In [1]:
using TensorFlow, PyPlot, Distributions
include(Pkg.dir("TensorFlow", "examples", "mnist_loader.jl"))

load_test_set (generic function with 2 methods)

In [2]:
function weight_variable(shape)
    initial = map(Float32, rand(Normal(0, .001), shape...))
    return Variable(initial)
end

function bias_variable(shape)
    initial = fill(Float32(.1), shape...)
    return Variable(initial)
end

function conv2d(x, W)
    nn.conv2d(x, W, [1, 1, 1, 1], "SAME")
end

function max_pool_2x2(x)
    nn.max_pool(x, [1, 2, 2, 1], [1, 2, 2, 1], "SAME")
end

max_pool_2x2 (generic function with 1 method)

In [3]:
loader = DataLoader()
session = Session(Graph())

2018-11-28 14:08:20.593211: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA


Session(Ptr{Void} @0x00007fa453239f90)

In [4]:
x = placeholder(Float32)
y_ = placeholder(Float32)

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = reshape(x, [-1, 28, 28, 1])

h_conv1 = nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = reshape(h_pool2, [-1, 7*7*64])
h_fc1 = nn.relu(h_pool2_flat * W_fc1 + b_fc1)

keep_prob = placeholder(Float32)
h_fc1_drop = nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = nn.softmax(h_fc1_drop * W_fc2 + b_fc2)

cross_entropy = reduce_mean(-reduce_sum(y_.*log(y_conv), axis=[2]))

train_step = train.minimize(train.AdamOptimizer(1e-4), cross_entropy)

#correct_prediction = indmax(y_conv, 2) .== indmax(y_, 2)
correct_prediction = equal(indmax(y_conv, 2), indmax(y_, 2))

accuracy = reduce_mean(cast(correct_prediction, Float32))

run(session, global_variables_initializer())

In [ ]:
for i in 1:1000
    batch = next_batch(loader, 50)
    if i%100 == 1
        train_accuracy = run(session, accuracy, Dict(x=>batch[1], y_=>batch[2], keep_prob=>1.0))
        info("step $i, training accuracy $train_accuracy")
    end
    run(session, train_step, Dict(x=>batch[1], y_=>batch[2], keep_prob=>.5))
end

INFO: step 1, training accuracy 0.14
INFO: step 101, training accuracy 0.9
INFO: step 201, training accuracy 0.92
INFO: step 301, training accuracy 0.9
INFO: step 401, training accuracy 0.94
INFO: step 501, training accuracy 0.98
INFO: step 601, training accuracy 0.98


In [ ]:
testx, testy = load_test_set()
test_accuracy = run(session, accuracy, Dict(x=>testx, y_=>testy, keep_prob=>1.0))
info("test accuracy $test_accuracy")

In [49]:
visualize()

2018-11-25 19:48:51.154371: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
TensorBoard 1.8.0 at http://scotty.pni.Princeton.EDU:6008 (Press CTRL+C to quit)
2018-11-25 19:48:53.818721: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
TensorBoard 1.8.0 at http://scotty.pni.Princeton.EDU:6009 (Press CTRL+C to quit)


START /usr/bin/firefox "http://localhost:6009/#graphs"
Running without a11y support!


Failed to open connection to "session" message bus: Using X11 for dbus-daemon autolaunch was disabled at compile time, set your DBUS_SESSION_BUS_ADDRESS instead
/usr/lib64/firefox/firefox: symbol lookup error: /lib64/libgtk-3.so.0: undefined symbol: g_log_structured_standard
xdg-open: no method available for opening 'http://localhost:6009/#graphs'


LoadError: [91mfailed process: Process(`xdg-open 'http://localhost:6009/#graphs'`, ProcessExited(3)) [3][39m